In [10]:
import numpy as np

from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split

#TODO: just make it a package
import sys
sys.path.append("..")

from tiny_xgboost import TinyXGBRegressor

In [11]:
def small_X_y_data(n_samples=5_000):
    """Small set of X, y data (single feature)"""

    def true_function(X):
        return np.sin(3 * X)

    def true_noise_scale(X):
        return np.abs(np.cos(X))

    np.random.seed(1234)
    X = np.random.uniform(-2, -1, n_samples)
    y = true_function(X) + np.random.normal(scale=true_noise_scale(X), size=n_samples)

    return X.reshape(-1,1), y


def gary_function(n_samples=5_000):
    
    X = np.random.uniform(0,1, n_samples)
    y_mean = 3*X +2 + 4*np.where(X>0.4,1,0)
    noise_size= y_mean/10
    
    outcomes = y_mean + np.random.normal(0, noise_size, n_samples)
    
    return X[..., np.newaxis], outcomes


In [12]:
#X, y = small_X_y_data(1_500)
X, y = gary_function(10_000)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, test_size=0.5)

# Before Running Tiny XGB, let's run homoskedastic XGB to get a good idea for what the log-likelihood to beat is

Loss is given by

$$\frac{\sigma ^{2}}{2}\left[\sum_{i=1}^{N}(y_{i} - f(x_{i}))^{2}\right] + N \ln \sigma$$


Once we have the residuals, we can just call these $r_{i}$ so we have

$$\frac{\sigma ^{2}}{2}\left[\sum_{i=1}^{N}r_{i}^{2}\right] + N \ln \sigma$$

which, when differentiated wrt $\gamma$ gives us


$$\sigma ^{*}= \sqrt{\frac{\sum_{i=1}^{N}r_{i}^{2}}{N}}$$

which makes intuitive sense, i.e. if the irreducible variance is larger, we expect the residuals to be larger as the model won't be able to fit the data as closely.

So now we plug this into our formula for the log test cross entropy (negative test log-likelihood)

In order to calculate test-log-likelihood, it's essentially the same formula, except that we must take care that $\sigma^{*}$ is calculated on the training data whereas the residuals are now test residuals, thus the end formula is given by

$$\frac{1}{2\left(\sigma^{*}\right)^{2}}\sum_{i=1}^{N_{test}}\left(f(x_{i})-y_{i}\right)^{2} + N_{test}\cdot\ln \sigma^{*}$$

In [13]:
from xgboost import XGBRegressor

In [50]:
def test_cross_entropy(sigma, residuals):

    return len(residuals)*np.log(sigma) + (0.5/(sigma*sigma))*np.sum(np.power(residuals, 2))

for MD in [1,2,3,4,5]:

    reg = XGBRegressor(max_depth=MD, learning_rate=0.1, objective='reg:squarederror', n_estimators=20_000, early_stopping_rounds=5, eval_metric='rmse')

    reg.fit(X_train, y_train, eval_set=[(X_test, y_test)], verbose=False)

    train_residuals = reg.predict(X_train) - y_train

    sigma_star = np.sqrt(np.mean(np.power(train_residuals, 2)))

    test_preds = reg.predict(X_test)
    test_residuals = test_preds - y_test
    print("Max depth = ", MD)
    print("sigma_star = ", sigma_star)
    print("Test cross entropy = ", test_cross_entropy(sigma_star, test_residuals))
    print("`Normalised` Test cross entropy = ", (1/len(test_residuals))*test_cross_entropy(sigma_star, test_residuals))

Max depth =  1
sigma_star =  0.650669364198992
Test cross entropy =  319.18020796561814
`Normalised` Test cross entropy =  0.06383604159312363
Max depth =  2
sigma_star =  0.6480970236192768
Test cross entropy =  320.9083884969723
`Normalised` Test cross entropy =  0.06418167769939447
Max depth =  3
sigma_star =  0.6450593660662713
Test cross entropy =  324.19933096349405
`Normalised` Test cross entropy =  0.06483986619269881
Max depth =  4
sigma_star =  0.6344358208477924
Test cross entropy =  330.19273996320817
`Normalised` Test cross entropy =  0.06603854799264164
Max depth =  5
sigma_star =  0.625923967707416
Test cross entropy =  347.40858520151414
`Normalised` Test cross entropy =  0.06948171704030283


# So the GW test-cross entropy is giving values of around 0.07 which should set the lengthscale to beat

In [51]:

model = TinyXGBRegressor(
    objective="distribution:normal",
    max_depth=2,
    n_estimators=10000,
    early_stopping_rounds=5,
    learning_rate=0.01,
)
model.fit(
    X_train,
    y_train,
    eval_set=(X_test, y_test),
    verbose=10,
)
preds = model.predict(X_test)
loc, scale = preds[:, 0], preds[:, 1]

[0]	train-loss=8.11743, val-loss=8.15618
[1]	train-loss=8.03464, val-loss=8.07339
[2]	train-loss=7.95475, val-loss=7.99358
[3]	train-loss=7.87775, val-loss=7.91662
[4]	train-loss=7.80356, val-loss=7.84235
[5]	train-loss=7.73228, val-loss=7.77114
[6]	train-loss=7.66376, val-loss=7.70256
[7]	train-loss=7.59811, val-loss=7.63703
[8]	train-loss=7.53519, val-loss=7.57408
[9]	train-loss=7.47513, val-loss=7.51421
[10]	train-loss=7.41770, val-loss=7.45652
[11]	train-loss=7.36320, val-loss=7.40219
[12]	train-loss=7.31125, val-loss=7.34995
[13]	train-loss=7.26229, val-loss=7.30120
[14]	train-loss=7.21598, val-loss=7.25498
[15]	train-loss=7.17254, val-loss=7.21187
[16]	train-loss=7.13164, val-loss=7.17069
[17]	train-loss=7.09377, val-loss=7.13315
[18]	train-loss=7.05842, val-loss=7.09744
[19]	train-loss=7.02614, val-loss=7.06561
[20]	train-loss=6.99637, val-loss=7.03553
[21]	train-loss=6.96978, val-loss=7.00940
[22]	train-loss=6.94564, val-loss=6.98503
[23]	train-loss=6.92481, val-loss=6.96471
[2

# Do these val losses look right? 

Here, $\sigma(x_{i}) = e^{-\gamma (x_{i})}$

Thus by my reckoning the log-likelihood is given by

$$L=\prod_{i=1}^{N}\frac{1}{\sqrt{2\pi}}e^{\gamma(x_{i})}e^{-\frac{e^{2\gamma(x_{i})}}{2}\left(y_{i}-f(x_{i})\right)^{2}}$$

so after taking the log and disgarding constants, we have

$$\sum_{i=1}^{N}\left[\gamma(x_{i})-\frac{1}{2}e^{2\gamma(x_{i})}\left(y_{i}-f(x_{i})\right)^{2}\right]$$

and thus the cross-entropy is given by the negative of this

$$\sum_{i=1}^{N}\left[\frac{1}{2}e^{2\gamma(x_{i})}\left(y_{i}-f(x_{i})\right)^{2}-\gamma(x_{i})\right]$$

In [55]:
def test_cross_entropy(regressor, x, y):
    preds = model.predict(x)
    y_preds, gamma_preds = preds[:, 0], preds[:, 1]
    
    residuals = np.power(y_preds - y, 2)
    
    return 0.5*np.dot(np.exp(2*gamma_preds), residuals) - np.sum(gamma_preds)

print(test_cross_entropy(model, X_test, y_test))
print(test_cross_entropy(model, X_test, y_test)/len(X_test))

1222602.9001742941
244.52058003485882
